In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.autograd import Variable
from torch import Tensor, optim, nn


In [2]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = pd.read_pickle(data_dir)
        self.labels = self.data['label']
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx, 1:].values
        label = self.labels.iloc[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

In [3]:
train_dataloader = DataLoader(StaticDataset('../data/processed_static_dataset.pkl'), batch_size=64, shuffle=True)
#test_dataloader = DataLoader(StaticDataset('data/test.pkl'), batch_size=64, shuffle=True)

In [5]:
print(f'PyTorch version: {torch.__version__}')
print('*' * 10)
print(f'_CUDA version: ')
!nvcc --version
print('*' * 10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.1.0
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:42:34_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080


In [9]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
def train(learning_rate, hidden_size, X_train, Y_train, dropout_prob=0.2, num_epochs=10):
    N = len(X_train)
    batch_size = 64
    num_batches = N // batch_size
    input_size = X_train.shape[1]
    model = NeuralNetwork(input_size, hidden_size, 2, dropout_prob).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        for i in range(num_batches):
            X_batch = X_train[i * batch_size:(i + 1) * batch_size]
            Y_batch = Y_train[i * batch_size:(i + 1) * batch_size]
            X_batch = torch.from_numpy(X_batch).float().to(device)
            Y_batch = torch.from_numpy(Y_batch).long().to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = loss_fn(output, Y_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1}/{num_epochs} finished")
    return model

In [13]:
X_train = pd.read_pickle('../data/processed_static_dataset.pkl').iloc[:, 1:].values
X_train.head()

,participant1_tier,participant1_leaguePoints,participant1_veteran,participant1_inactive,participant1_freshBlood,participant1_hotStreak,participant1_champion_lastPlayTime,participant1_champion_championLevel,participant1_champion_championPoints,participant1_champion_championPointsSinceLastLevel,...,participant1_winrate,participant2_winrate,participant3_winrate,participant4_winrate,participant5_winrate,participant6_winrate,participant7_winrate,participant8_winrate,participant9_winrate,participant10_winrate
0,7.1,207,0,0,1,0,1.696710e+12,2.0,3837.0,2037.0,...,0.521912,0.645161,0.510582,0.542088,0.727273,0.555556,0.722222,0.516588,0.498476,0.333333
1,6.3,56,0,0,0,0,NaN,NaN,NaN,NaN,...,0.512953,0.645161,0.494505,0.497297,0.556098,0.666667,0.546326,0.513382,0.533654,0.505236
2,6.3,29,0,0,0,0,1.696428e+12,7.0,592841.0,571241.0,...,0.583333,0.647059,0.493976,0.809524,0.542522,0.497696,0.645161,0.507868,0.530233,0.562500
3,6.2,29,0,0,1,0,1.696905e+12,5.0,482286.0,460686.0,...,0.565789,0.707317,0.650794,0.623377,0.583333,0.645161,0.503185,0.511450,0.375000,0.600000
4,2.4,29,0,0,0,0,1.696928e+12,7.0,106023.0,84423.0,...,0.571429,0.547368,0.636364,0.564516,0.504132,0.777778,0.488889,0.598958,0.509091,0.625000


In [8]:
train(0.001, 64, X_train, Y_train, num_epochs=10)